# 1、安装依赖库

In [5]:
!pip install tensorflow
!pip install keras
!pip install keras-rl
!pip install PyOpenGL
!pip install pyglet==1.5.11
!pip install gym

In [6]:
import tensorflow as tf
import gym
import pyglet
print(tf.__version__)
print(gym.__version__)
print(pyglet.version)

2.6.0
0.19.0
1.5.11


# 2、测试Gym环境

In [10]:
import gym
env = gym.make("CartPole-v1")
observation = env.reset()
import random
for _ in range(1000):
  env.render()
#   action = env.action_space.sample() # your agent here (this takes random actions)
  action = random.choice([0,1])
  observation, reward, done, info = env.step(action)

  if done:
    observation = env.reset()
env.close()

# 3、使用keras-rl建立强化学习模型

In [11]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n
status = env.observation_space.shape

def build_model(status,nb_actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + status))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [12]:
model = build_model(status,nb_actions)

In [15]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
def build_agent(nb_actions,model):
    memory = SequentialMemory(limit=50000, window_length=1)
    policy = BoltzmannQPolicy()
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
                   target_model_update=1e-2, policy=policy)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    return dqn

In [16]:
dqn =  build_agent(nb_actions,model)

In [7]:
dqn.fit(env, nb_steps=10000, visualize=True, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)


/Users/liangsy/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    8/10000 [..............................] - ETA: 2:40 - reward: 1.0000  

/Users/liangsy/opt/anaconda3/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
done, took 52.914 seconds


In [9]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [10]:
dqn.test(env, nb_episodes=20, visualize=True)

Testing for 20 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 175.000, steps: 175
Episode 6: reward: 191.000, steps: 191
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 199.000, steps: 199
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 177.000, steps: 177
Episode 16: reward: 173.000, steps: 173
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 169.000, steps: 169
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200


In [18]:
del model
del dqn
model = build_model(status,nb_actions)
dqn =  build_agent(nb_actions,model)
dqn.load_weights('dqn_weights.h5f')

In [19]:
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 189.000, steps: 189
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 180.000, steps: 180
Episode 8: reward: 199.000, steps: 199
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
